In [121]:
import pandas as pd

### 1. concat 데이터 불러오기

17487 rows × 140 columns

In [122]:
df = pd.read_csv('../dataset_kosdaq/kosdaq_konex_concat.csv',index_col=0)

C:\Users\hotte\AppData\Local\Temp\ipykernel_13128\1743574232.py:1: DtypeWarning: Columns (137) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset_kosdaq/kosdaq_konex_concat.csv',index_col=0)


### 2. 재무비율 생성을 위해 2년 연속 재무제표가 존재하지 않으면 제거

17456 rows × 140 columns

In [123]:
# 회계년도를 시간 데이터로 변경
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')

# 거래소코드와 회계년도 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

# 3년 연속 데이터 체크
def has_consecutive_years(group):
    # year 만 가져오기
    years = group['회계년도'].dt.year
    # 현재 year에서 전 yeear을 뺀값이 1, 총합이 2이상인 것을 반환
    return (years.diff().fillna(1) == 1).cumsum().max() >= 2

# 거래소코드를 기준으로 groupby 후 filter
df = df.groupby('거래소코드').filter(has_consecutive_years)

### 3. Labeling 진행
상장폐지사유 데이터 불러오기

In [124]:
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

# 감사의견코드 중 DI, DS DU, AG, QS가 존재하면 A 아니면 a
df['감사의견_y'] = df['감사의견코드'].apply(lambda x: "A" if x in ['DI', 'DS', 'DU', 'AG', 'QS'] else "a")
# 회계년도를 datetime
df['회계년도'] = pd.to_datetime(df['회계년도'], format='%Y-%m-%d')
df = df.sort_values(by=['거래소코드', '회계년도'])

df['자본잠식률_100'] = df['자본잠식률'].apply(lambda x: "B" if x >= 100 else "b")
# 데이터셋을 거래소코드와 회계년도 기준으로 정렬
df.sort_values(by=['거래소코드', '회계년도'], inplace=True)

# 새로운 'Label' 컬럼을 기본값 'c'로 추가
df['자본잠식률_50'] = 'c'

# 이전 행의 값 초기화
prev_code = None
prev_cap_erode_rate = None

# 각 행을 순회하며 조건에 따라 'Label' 컬럼 설정
for index, row in df.iterrows():
    current_code = row['거래소코드']
    current_cap_erode_rate = row['자본잠식률']

    # 현재 행과 이전 행이 같은 회사이고, 두 해 모두 자본잠식률이 50 이상인 경우 'C'로 표시
    if current_code == prev_code and current_cap_erode_rate >= 50 and prev_cap_erode_rate >= 50:
        df.at[index, '자본잠식률_50'] = 'C'

    # 이전 행 값 업데이트
    prev_code = current_code
    prev_cap_erode_rate = current_cap_erode_rate

# 라벨링 조건에 따라 'y' 컬럼 생성
df['y'] = ((df['감사의견_y'] == 'A') | 
        (df['자본잠식률_100'] == 'B') | 
        (df['자본잠식률_50'] == 'C')).astype(int)

In [125]:
label = pd.read_csv('../dataset_kosdaq/상장폐지사유.csv')

In [126]:
label_list = ['감사의견 거절(감사범위 제한)','상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소', "상장폐지 신청('23.06.28)", '감사의견 거절(감사범위 제한 및 계속기업가정 불확실성)',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력 불확실성)',
    '기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정',
    '상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동사유 미해소', '감사의견 거절',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
    '해산사유 발생(파산선고)', '감사의견 한정(감사범위 제한)',
    '법정제출기한까지 사업보고서를 제출하지 아니한 후, 법정제출기한의 다음날부터 10일이내에 사업보고서를 제출하지 아니함',
    '기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당',
    '최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '감사의견거절(감사범위 제한)', '감사의견거절(감사범위제한)',
    '감사의견거절(감사범위제한 및 계속기업 존속 불확실)',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조제1항에 따른 사업보고서 법정제출기한 내 미제출, 최근 2년간 3회 이상 공시규정 제19조제1항의 규정에 의한 사업보고서, 반기보고서 또는 분기보고서 법정제출기한 내 미제출',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동사유를 미해소',
    '제28조제1항제9호에 따라 관리종목 지정 후 공시규정 제19조 제1항에 따른 분기보고서 법정기한 내 미제출',
    '감사의견거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '감사의견거절(범위제한 및 계속기업 존속능력에 대한 불확실성)',
    "'16사업연도 감사의견거절(계속기업 존속능력에 대한 불확실성) 및 '17사업연도 반기 감사의견거절(계속기업 존속능력에 대한 불확실성)",
    '감사의견 거절(내부회계관리제도상의 취약점, 계속기업가정의 불확실성 및 재고자산 관련 감사범위 제한)',
    '감사범위제한으로 인한 한정의견', '자본전액잠식 등',
    '최근 5사업연도 연속 영업손실 발생',
    '감사의견 거절(감사범위 제한 및 기업회계기준 위배)', '2반기 연속 자본잠식률 50% 이상',
    '감사의견 거절(감사범위 제한으로 인한 의견거절)',
    "'14사업연도 자본잠식률 50%이상 사유로 관리종목 지정 후 '15사업연도 반기 감사의견 거절", '자본전액잠식',
    '기업의 계속성, 경영의 투명성 또는 그 밖에 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지 기준에 해당한다고 결정',
    '감사의견 거절(계속기업 존속능력에 대한 불확실성 및 감사범위 제한)',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래 정지',
    "상장폐지 신청('15.01.15)", '감사의견 거절(계속기업 존속능력에 대한 불확실성)',
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성', "상장폐지신청('14.04.22)",
    '최종부도 발생', '자본전액잠식등', '최근 5사업연도 연속 영업손실 발생 등',
    '기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려하여 상장폐지가 필요하다고 인정',
    "상장폐지신청('13.09.06)", '의견거절(감사범위 제한)',
    '감사의견거절(감사범위 제한 및 계속기업으로서의 존속능력에 대한 불확실성)', "상장폐지 신청('13.05.08)",
    '감사의견거절(계속기업으로서의 존속능력에 대한 불확실성)', '감사의견 부적정',
    '반기 재무제표에 대한 검토의견거절로 관리종목 지정후 자본잠식률 50% 이상', '2반기 연속 자본잠식률 50%이상',
    '상장예비심사 청구서 미제출로 관리종목 지정후 1개월 이내 동 사유 미해소','외부감사인의 감사의견 의견거절',
    '발행한 어음 또는 수표가 주거래은행에 의하여 최종부도로 결정되거나 거래은행에 의한 거래정지',
    '감사의견 의견거절(감사범위제한 및 계속기업 존속불확실성)', 
    '외부감사인의 감사의견 거절(감사범위 제한), 감사의견 한정(감사범위 제한)',
    '외부감사인의 감사의견 의견거절(감사범위 제한)', '감사의견 거절(계속기업 존속능력에 대한 불확실성)',
    '파산선고에 따른 해산사유 발생',
    '감사의견 거절(감사범위 제한)', 
    '감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)',
    '최종부도 발생','감사의견 거절(감사범위제한)']

In [127]:
# Filtering the dataset for rows where 폐지사유 is in the label list
filtered_data = label[label['폐지사유'].isin(label_list)]

# Selecting only the 종목코드 and 폐지일자 columns
result = filtered_data[['종목코드', '폐지일자']]


In [128]:
import pandas as pd

# df1에서 '종목코드'를 기준으로 그룹화하고 '회계년도'를 오름차순으로 정렬합니다.
df1_sorted = df.groupby('거래소코드').apply(lambda x: x.sort_values('회계년도', ascending=True)).reset_index(drop=True)

# df2의 '폐지일자'를 datetime 형식으로 변환하고 '폐지년'을 추출합니다.
result['폐지일자'] = pd.to_datetime(result['폐지일자'])
result['폐지년'] = result['폐지일자'].dt.year
result['폐지전년'] = result['폐지년'] - 1  # '폐지년'의 전년도를 계산합니다.

# df1에 '회계년' 컬럼을 추가합니다.
df['회계년'] = pd.to_datetime(df['회계년도']).dt.year

# df1의 각 행에 대해 df2의 '폐지전년'과 일치하는 '회계년'이 있는지 확인하고, 
# 일치하는 경우 해당 행의 'y' 라벨을 1로 설정합니다.
delisting_dict = result.set_index('종목코드')['폐지전년'].to_dict()
df['D'] = df.apply(lambda row: "D" if delisting_dict.get(row['거래소코드'], 0) == row['회계년'] else "d", axis=1)

# Removing the additional columns (except 'y') that were added during the process
columns_to_remove = ['회계년', '폐지전년','폐지일자','폐지년']
df1_cleaned = df.drop(columns=columns_to_remove, errors='ignore')


C:\Users\hotte\AppData\Local\Temp\ipykernel_13128\1715330378.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['폐지일자'] = pd.to_datetime(result['폐지일자'])
C:\Users\hotte\AppData\Local\Temp\ipykernel_13128\1715330378.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['폐지년'] = result['폐지일자'].dt.year
C:\Users\hotte\AppData\Local\Temp\ipykernel_13128\1715330378.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [129]:
df = df1_cleaned

In [130]:
# 라벨링 조건에 따라 'y' 컬럼 생성
df['y'] = ((df['감사의견_y'] == 'A') | 
        (df['자본잠식률_100'] == 'B') | 
        (df['자본잠식률_50'] == 'C') |
        (df['자본잠식률_50'] == 'D')).astype(int)

In [131]:
df['y']

13164    0
13165    0
13166    0
13167    0
13168    0
        ..
3674     0
464      0
465      0
466      0
467      0
Name: y, Length: 17456, dtype: int32

### 4. 부도년도 이후 데이터 삭제

15246 rows × 150 columns

In [132]:
def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 작거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] <= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

In [133]:
df[(df['y']==1)&(df['자본잠식률'].isna())]

,회사명,거래소코드,회계년도,이자보상배율,영업이익증가율,총자본증가율,자기자본증가율,순이익증가율,자본잠식률,산업코드,...,부가가치율,노동소득분배율,자본분배율,이윤분배율,감사의견,감사의견_y,자본잠식률_100,자본잠식률_50,y,D
13634,(주)네오셈,253590,2018-12-01,NaN,NaN,NaN,NaN,NaN,NaN,117201,...,NaN,NaN,NaN,NaN,NaN,a,b,C,1,d
14017,(주)나인테크,267320,2019-12-01,NaN,NaN,NaN,NaN,NaN,NaN,117201,...,NaN,NaN,NaN,NaN,NaN,a,b,C,1,d


### 5. 부도년도 이전 데이터

전년도의 자본잠식률이 50%면 1
나머지 데이터는 삭제

182114 rows × 156 columns

In [134]:
# 거래소 코드와 회계년도를 기준으로 정렬
df = df.sort_values(by=['거래소코드', '회계년도'])

for i in range(1, len(df)):
    # y가 1 인 경우
    if df.loc[i, 'y'] == 1:
        # 해당 행과 전 행의 거래소 코드가 같으면
        if df.loc[i, '거래소코드'] == df.loc[i-1, '거래소코드']:
            # 전 행의 자본잠식률이 50이상이면
            if df.loc[i-1, '자본잠식률'] >= 50:
                # 전 행의 y 가 1
                df.loc[i-1, 'y'] = 1

In [135]:
# 부도 데이터 중 정상 데이터 삭제

def remove_post_default_data(group):
    # 최초 부도 년도 설정
    default_year = group[group['y'] == 1]['회계년도'].min()
    
    # default 값이 nan이 아니면 크거나 같은 데이터만 가져옴
    if not pd.isnull(default_year):
        group = group[group['회계년도'] >= default_year]
    
    return group

# 회사명으로 그룹바이 후 함수 적용
df = df.groupby('거래소코드').apply(remove_post_default_data).reset_index(drop=True)

In [138]:
df[df['y'] == 1]

,회사명,거래소코드,회계년도,이자보상배율,영업이익증가율,총자본증가율,자기자본증가율,순이익증가율,자본잠식률,산업코드,...,부가가치율,노동소득분배율,자본분배율,이윤분배율,감사의견,감사의견_y,자본잠식률_100,자본잠식률_50,y,D
158,울트라건설(주),4320,2014-12-01,-13.498918,0.00,-48.045405,-202.895290,-2750.187665,2.939162e+02,64101,...,-35.48,0.0,0.00,0.00,"당기순손실 154,671백만원 발생, 유동부채가 유동자산을 135,612백만원 초과...",A,B,c,1,D
192,(주)렉스엘이앤지,4790,2012-12-01,-17.412302,0.00,NaN,NaN,NaN,9.781426e+01,32605,...,-160.19,0.0,0.00,0.00,"15,968만원의 영업손실과 18,920백만원의 당기순손실 발생, 유동부채가 유동자...",a,b,c,1,d
193,(주)렉스엘이앤지,4790,2013-12-01,-9.229252,0.00,-57.019962,-1068.124911,76.067550,2.157063e+02,32605,...,-79.60,0.0,0.00,0.00,2013년 8월 30일부터 회사의 주식은 한국거래소 코스닥시장에서 주된 영업의 정지...,A,B,C,1,D
447,(주)행남사,8800,2016-12-01,-20.753142,-1542.64,-36.854215,-55.597528,-7277.338543,5.555532e+01,32302,...,-69.14,0.0,0.00,0.00,"당기순손실이 18,924백만원이며, 당기말 현재 결손금이 23,142백만원이고 자본...",a,b,c,1,d
448,(주)행남사,8800,2017-12-01,-11.011284,0.00,-5.769514,72.531228,20.211687,5.363649e+01,32302,...,-89.98,0.0,0.00,0.00,NaN,a,b,C,1,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14591,차이나그레이트스타인터내셔널리미티드,900040,2018-12-01,7.471800,0.00,2.101596,6.240733,295.152821,-3.394200e+03,137105,...,15.40,18.52,81.48,45.26,NaN,A,b,c,1,d
14603,에스앤씨엔진그룹리미티드,900080,2019-12-01,30.439012,-34.36,-0.733638,1.080114,-68.280181,-3.131925e+02,137105,...,37.44,25.84,74.16,13.70,NaN,A,b,c,1,d
14637,완리인터내셔널홀딩스,900180,2017-12-01,inf,-28.28,-2.484225,-5.481293,-13.717312,-2.265860e+02,137105,...,14.72,0.0,0.00,74.58,NaN,A,b,c,1,D
14710,코오롱티슈진(주),950160,2017-12-01,-59.941506,0.00,355.958406,640.597335,-721.553661,-6.266310e+06,32101,...,-1061.25,0.0,0.00,0.00,NaN,A,b,c,1,d


### 6. 중간 데이터 확인
#전체 182114
#정상 171409
#부도 10705

In [139]:
df

,회사명,거래소코드,회계년도,이자보상배율,영업이익증가율,총자본증가율,자기자본증가율,순이익증가율,자본잠식률,산업코드,...,부가가치율,노동소득분배율,자본분배율,이윤분배율,감사의견,감사의견_y,자본잠식률_100,자본잠식률_50,y,D
0,삼천당제약(주),250,2012-12-01,45.838520,0.00,NaN,NaN,NaN,-7.663424e+02,32102,...,30.97,68.99,31.01,5.65,NaN,a,b,c,0,d
1,삼천당제약(주),250,2013-12-01,8.274610,166.42,27.277086,37.626249,466.163338,-1.092315e+03,32102,...,33.21,60.04,39.96,21.86,NaN,a,b,c,0,d
2,삼천당제약(주),250,2014-12-01,34.604397,19.31,11.013223,17.147234,127.488875,-1.226908e+03,32102,...,39.99,51.54,48.46,36.30,NaN,a,b,c,0,d
3,삼천당제약(주),250,2015-12-01,126.758744,9.61,6.823177,10.480341,3.536727,-1.353253e+03,32102,...,39.04,53.24,46.76,34.67,NaN,a,b,c,0,d
4,삼천당제약(주),250,2016-12-01,126.362668,17.32,11.317289,10.814120,-12.648631,-1.454533e+03,32102,...,39.88,54.09,45.91,26.56,NaN,a,b,c,0,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14716,(주)소마젠,950200,2019-12-01,-10.899532,NaN,NaN,NaN,NaN,1.410593e+03,137309,...,NaN,NaN,NaN,NaN,NaN,a,B,c,1,d
14717,(주)네오이뮨텍,950220,2019-12-01,-108.221346,NaN,NaN,NaN,NaN,-6.076765e+06,137001,...,NaN,NaN,NaN,NaN,NaN,a,b,c,0,d
14718,(주)네오이뮨텍,950220,2020-12-01,-163.925632,0.00,-21.129085,-27.319255,-114.512509,-4.676562e+06,137001,...,0.00,0.0,0.00,0.00,NaN,a,b,c,0,d
14719,(주)네오이뮨텍,950220,2021-12-01,-239.478068,0.00,95.543438,92.388480,-72.320502,-6.930714e+06,137001,...,0.00,0.0,0.00,0.00,NaN,a,b,c,0,d


In [140]:
df[df['y']==1]

,회사명,거래소코드,회계년도,이자보상배율,영업이익증가율,총자본증가율,자기자본증가율,순이익증가율,자본잠식률,산업코드,...,부가가치율,노동소득분배율,자본분배율,이윤분배율,감사의견,감사의견_y,자본잠식률_100,자본잠식률_50,y,D
158,울트라건설(주),4320,2014-12-01,-13.498918,0.00,-48.045405,-202.895290,-2750.187665,2.939162e+02,64101,...,-35.48,0.0,0.00,0.00,"당기순손실 154,671백만원 발생, 유동부채가 유동자산을 135,612백만원 초과...",A,B,c,1,D
192,(주)렉스엘이앤지,4790,2012-12-01,-17.412302,0.00,NaN,NaN,NaN,9.781426e+01,32605,...,-160.19,0.0,0.00,0.00,"15,968만원의 영업손실과 18,920백만원의 당기순손실 발생, 유동부채가 유동자...",a,b,c,1,d
193,(주)렉스엘이앤지,4790,2013-12-01,-9.229252,0.00,-57.019962,-1068.124911,76.067550,2.157063e+02,32605,...,-79.60,0.0,0.00,0.00,2013년 8월 30일부터 회사의 주식은 한국거래소 코스닥시장에서 주된 영업의 정지...,A,B,C,1,D
447,(주)행남사,8800,2016-12-01,-20.753142,-1542.64,-36.854215,-55.597528,-7277.338543,5.555532e+01,32302,...,-69.14,0.0,0.00,0.00,"당기순손실이 18,924백만원이며, 당기말 현재 결손금이 23,142백만원이고 자본...",a,b,c,1,d
448,(주)행남사,8800,2017-12-01,-11.011284,0.00,-5.769514,72.531228,20.211687,5.363649e+01,32302,...,-89.98,0.0,0.00,0.00,NaN,a,b,C,1,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14591,차이나그레이트스타인터내셔널리미티드,900040,2018-12-01,7.471800,0.00,2.101596,6.240733,295.152821,-3.394200e+03,137105,...,15.40,18.52,81.48,45.26,NaN,A,b,c,1,d
14603,에스앤씨엔진그룹리미티드,900080,2019-12-01,30.439012,-34.36,-0.733638,1.080114,-68.280181,-3.131925e+02,137105,...,37.44,25.84,74.16,13.70,NaN,A,b,c,1,d
14637,완리인터내셔널홀딩스,900180,2017-12-01,inf,-28.28,-2.484225,-5.481293,-13.717312,-2.265860e+02,137105,...,14.72,0.0,0.00,74.58,NaN,A,b,c,1,D
14710,코오롱티슈진(주),950160,2017-12-01,-59.941506,0.00,355.958406,640.597335,-721.553661,-6.266310e+06,32101,...,-1061.25,0.0,0.00,0.00,NaN,A,b,c,1,d


In [133]:
df.to_csv('../dataset_kosdaq/상장최종.csv')